# Chapter 1

At the center of spaCy is the object containing the processing pipeline. We usually call this variable "nlp".

For example, to create an English nlp object, you can import the English language class from `spacy.lang.en` and instantiate it. You can use the nlp object like a function to analyze text. It contains all the different components in the pipeline. It also includes language-specific rules used for tokenizing the text into words and punctuation. spaCy supports a variety of languages that are available in `spacy.lang`.

In [1]:
from spacy.lang.en import English
nlp = English()

When you process a text with the nlp object, spaCy creates a Doc object – short for "document". The Doc lets you access information about the text in a structured way, and no information is lost. The Doc behaves like a normal Python sequence by the way and lets you iterate over its tokens, or get a token by its index. But more on that later!

In [2]:
doc = nlp("Hello world!")
for token in doc:
    print(token.text)

Hello
world
!


Documents are instances of the class `Doc` and are composed of tokens, of class `Token`.

In [3]:
type(doc), type(token)

(spacy.tokens.doc.Doc, spacy.tokens.token.Token)

**Token objects** represent the tokens in a document – for example, a word or a punctuation character. To get a token at a specific position, you can index into the Doc. Token objects also provide various attributes that let you access more information about the tokens. For example, the dot text attribute returns the verbatim token text.

In [4]:
print(doc[1], type(doc[1]))
print(doc[1].text, type(doc[1].text))

world <class 'spacy.tokens.token.Token'>
world <class 'str'>


A **Span object** is a slice of the document consisting of one or more tokens. It's only a view of the Doc and doesn't contain any data itself. To create a Span, you can use Python's slice notation. For example, 1 colon 3 will create a slice starting from the token at position 1, up to – but not including! – the token at position 3. Our span can be longer than the doc object itself, but this is not an issue.

In [5]:
print(doc[1:2], doc[1:10])
print(type(doc[1:2]))

world world!
<class 'spacy.tokens.span.Span'>


Span object, like token objects, have a `.text` attribute.

In [6]:
doc[:2].text

'Hello world'

Here you can see some of the available token attributes:

- "i" is the index of the token within the parent document.
- "text" returns the token text.
- "is alpha", "is punct" and "like num" return boolean values indicating whether the token consists of alphanumeric characters, whether it's punctuation or whether it resembles a number. For example, a token "10" – one, zero – or the word "ten" – T, E, N.

These attributes are also called **lexical attributes**: they refer to the entry in the vocabulary and don't depend on the token's context.

In [7]:
doc = nlp("It costs $5.")

print('Index:   ', [token.i for token in doc])
print('Text:    ', [token.text for token in doc])
print('is_alpha:', [token.is_alpha for token in doc])
print('is_punct:', [token.is_punct for token in doc])
print('like_num:', [token.like_num for token in doc])

Index:    [0, 1, 2, 3, 4]
Text:     ['It', 'costs', '$', '5', '.']
is_alpha: [True, True, False, False, False]
is_punct: [False, False, False, False, True]
like_num: [False, False, False, True, False]


In [8]:
doc = nlp("I like tree kangaroos and narwhals.")
print(doc[0], doc[0].text)
print(doc[2:4].text)
print(doc[2:6])

I I
tree kangaroos
tree kangaroos and narwhals


In [9]:
# Exercise 4

doc = nlp(
    "In 1990, more than 60% of people in East Asia were in extreme poverty. "
    "Now less than 4% are."
)

# Iterate over the tokens in the doc
for token in doc:
    # Check if the token resembles a number
    if token.like_num:
        # Get the next token in the document
        next_token = doc[token.i + 1]
        # Check if the next token's text equals '%'
        if next_token.text == "%":
            print("Percentage found:", token.text)

Percentage found: 60
Percentage found: 4


## Statistical Models

Some of the most interesting things you can analyze are context-specific: for example, whether a word is a verb or whether a span of text is a person name. Statistical models enable spaCy to make predictions in context. This usually includes part-of speech tags, syntactic dependencies and named entities. Models are trained on large datasets of labeled example texts. They can be updated with more examples to fine-tune their predictions – for example, to perform better on your specific data. 

SpaCy provides a number of pre-trained model packages you can download using the "spacy download" command. For example, the "en_core_web_sm" package is a small English model that supports all core capabilities and is trained on web text. The `spacy.load` method loads a model package by name and returns an nlp object. The package provides the binary weights that enable spaCy to make predictions. It also includes the vocabulary, and meta information to tell spaCy which language class to use and how to configure the processing pipeline. To download the language model for English, enter this command.

In order to use the language model above we need to enter the following commands.

In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

### Predicting Part-of-Speech

Let's take a look at the model's predictions. In this example, we're using spaCy to predict part-of-speech tags, the word types in context. First, we load the small English model and receive an nlp object. Next, we're processing the text "She ate the pizza". For each token in the Doc, we can print the text and the "pos underscore" attribute, the predicted part-of-speech tag. In spaCy, attributes that return strings usually end with an underscore – attributes without the underscore return an ID. Here, the model correctly predicted "ate" as a verb and "pizza" as a noun.

In [11]:
doc = nlp("She ate the pizza")

# Iterate over the tokens
for token in doc:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_)

She PRON
ate VERB
the DET
pizza NOUN


### Predicting syntactic dependencies

In addition to the part-of-speech tags, we can also predict how the words are related. For example, whether a word is the subject of the sentence or an object. The "dep underscore" attribute returns the predicted dependency label. The **head** attribute returns the syntactic head token. You can also think of it as the parent token this word is attached to.

In [12]:
for token in doc:
    print(token.head, token.pos_, token.dep_, token.head.text)

ate PRON nsubj ate
ate VERB ROOT ate
pizza DET det pizza
ate NOUN dobj ate


### Dependency label scheme

To describe syntactic dependencies, spaCy uses a standardized label scheme. Here's an example of some common labels: The pronoun "She" is a nominal subject attached to the verb – in this case, to "ate". The noun "pizza" is a direct object attached to the verb "ate". It is eaten by the subject, "she". The determiner "the", also known as an article, is attached to the noun "pizza".

### Predicting name entities

Named entities are "real world objects" that are assigned a name – for example, a person, an organization or a country. The `doc.ents` property lets you access the named entities predicted by the model. It returns an iterator of Span objects, so we can print the entity text and the entity label using the "label underscore" attribute. In this case, the model is correctly predicting "Apple" as an organization, "U.K." as a geopolitical entity and "$1 billion" as money.

In [13]:
doc = nlp(u"Apple is looking at buying U.K. startup for $1 billion")

# Iterate over the predicted entities
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

Apple ORG
U.K. GPE
$1 billion MONEY


However, this is unlikely to succeed with highly technical terms. In the example below, only the cardinal number is properly captured.

In [14]:
doc = nlp(u'Competition between Tamoxifen and Entresto while tuning a cobas 6000')
for ent in doc.ents:
    print(ent.text, ent.label_)

6000 CARDINAL


Some tags can have rather obscure names. The `.explain` method can help in this respect.

In [15]:
print(spacy.explain('GPE'))
print(spacy.explain('NNP'))
print(spacy.explain('dobj'))

Countries, cities, states
noun, proper singular
direct object


## Rule-based matching

In this lesson, we'll take a look at spaCy's matcher, which lets you write rules to find words and phrases in text.

Compared to regular expressions, the matcher works with Doc and Token objects instead of only strings. It's also more flexible: you can search for texts but also other lexical attributes. You can even write rules that use the model's predictions. For example, find the word "duck" only if it's a verb, not a noun.

**Match patterns** are lists of dictionaries. Each dictionary describes one token. The keys are the names of token attributes, mapped to their expected values. In this example, we're looking for two tokens with the text "iPhone" and "X". We can also match on other token attributes. Here, we're looking for two tokens whose lowercase forms equal "iphone" and "x". We can even write patterns using attributes predicted by the model. Here, we're matching a token with the lemma "buy", plus a noun. The lemma is the base form, so this pattern would match phrases like "buying milk" or "bought flowers".

To use a pattern, we first import the matcher from spacy dot matcher. We also load a model and create the nlp object. The matcher is initialized with the shared vocabulary, `nlp.vocab`. You'll learn more about this later – for now, just remember to always pass it in.

The matcher `.add` method lets you add a pattern. The first argument is a unique ID to identify which pattern was matched. The second argument is an optional callback. We don't need one here, so we set it to None. The third argument is the pattern. To match the pattern on a text, we can call the matcher on any doc. This will return the matches.

In [16]:
# import spacy

# Import the Matcher
from spacy.matcher import Matcher

# Load a model and create the nlp object
# nlp = spacy.load('en_core_web_sm')

# Initialize the matcher with the shared vocab
matcher = Matcher(nlp.vocab)

# Add the pattern to the matcher
pattern = [{'TEXT': 'iPhone'}, {'TEXT': 'X'}]
matcher.add('IPHONE_PATTERN', None, pattern)

# Process some text
doc = nlp("New iPhone X release date leaked")

# Call the matcher on the doc
matches = matcher(doc)
matches

[(9528407286733565721, 1, 3)]

When you call the matcher on a doc, it returns a list of tuples. Each tuple consists of three values: the match ID, the start index and the end index of the matched span. This means we can iterate over the matches and create a Span object: a slice of the doc at the start and end index.

In [17]:
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

iPhone X


Here's an example of a more complex pattern using lexical attributes. We're looking for five tokens:

- A token consisting of only digits.
- Three case-insensitive tokens for "fifa", "world" and "cup".
- And a token that consists of punctuation.

The pattern matches the tokens "2018 FIFA World Cup:".

In [18]:
pattern = [
    {'IS_DIGIT': True},
    {'LOWER': 'fifa'},
    {'LOWER': 'world'},
    {'LOWER': 'cup'},
    {'IS_PUNCT': True}
]

doc = nlp("2018 FIFA World Cup: France won!")

In this example, we're looking for two tokens: A verb with the lemma "love", followed by a noun. This pattern will match "loved dogs" and "love cats".

In [19]:
pattern = [
    {'LEMMA': 'love', 'POS': 'VERB'},
    {'POS': 'NOUN'}
]

doc = nlp("I loved dogs but now I love cats more.")

**Operators and quantifiers** let you define how often a token should be matched. They can be added using the "OP" key. Here, the "?" operator makes the determiner token optional, so it will match a token with the lemma "buy", an optional article and a noun.

In [20]:
pattern = [
    {'LEMMA': 'buy'},
    {'POS': 'DET', 'OP': '?'},  # optional: match 0 or 1 times
    {'POS': 'NOUN'}
]

doc = nlp("I bought a smartphone. Now I'm buying apps.")

"OP" can have one of four values:

- An "!" negates the token, so it's matched 0 times.
- A "?" makes the token optional, and matches it 0 or 1 times.
- A "+" matches a token 1 or more times.
- And finally, an "*" matches 0 or more times.

Operators can make your patterns a lot more powerful, but they also add more complexity – so use them wisely.


|             |                         | 
|-------------|-------------------------|
| {'OP': '!'} | Negation: match 0 times | 
| {'OP': '?'} |	Optional: match 0 or 1 times
| {'OP': '+'} |	Match 1 or more times
| {'OP': '*'} |	Match 0 or more times |

In [21]:
doc = nlp("New iPhone X release date leaked as Apple reveals pre-orders by mistake")
matcher = Matcher(nlp.vocab)
pattern = [
    {'TEXT': 'iPhone'},
    {'TEXT': 'X'}
]
matcher.add('IPHONE_X_PATTERN', None, pattern)
matches = matcher(doc)

for match_id, start, end in matches:
    print(doc[start:end])

iPhone X


A possibly nicer way to run the last code snippet, especially when there are multiple matches is:

In [24]:
def print_matches(matches):
    print('Matches: ', [doc[start:end] for match_id, start, end in matches])

In [26]:
doc = nlp(
    "After making the iOS update you won't notice a radical system-wide "
    "redesign: nothing like the aesthetic upheaval we got with iOS 7. Most of "
    "iOS 11's furniture remains the same as in iOS 10. But you will discover "
    "some tweaks once you delve a little deeper."
)

pattern = [
    {'TEXT': 'iOS'},
    {'IS_DIGIT': True}
]

matcher.add('IOS_PATTERN', None, pattern)
matches = matcher(doc)

print_matches(matches)

Matches:  [iOS 7, iOS 11, iOS 10]


Let's find different forms of 'download' plus a proper noun.

In [29]:
doc = nlp(
    "i downloaded Fortnite on my laptop and can't open the game at all. Help? "
    "so when I was downloading Minecraft, I got the Windows version where it "
    "is the '.zip' folder and I used the default program to unpack it... do "
    "I also need to download Winzip?"
)

pattern = [
    {'LEMMA': 'download'}, 
    {'POS': 'PROPN'} # Note, 'PROPN', not 'NOUN'
]

matcher.add('DOWNLOAD_THINGS_PATTERN', None, pattern)
matches = matcher(doc)

print_matches(matches)

Matches:  [downloaded Fortnite, downloading Minecraft, download Winzip]


Write one pattern that matches adjectives ('ADJ') followed by one or two 'NOUN's (one noun and one optional noun).

In [30]:
doc = nlp(
    "Features of the app include a beautiful design, smart search, automatic "
    "labels and optional voice responses."
)

pattern = [
    {'POS': 'ADJ'},
    {'POS': 'NOUN'},
    {'POS': 'NOUN', 'OP': '?'}
]

matcher.add('ADJ_NOUN_PATTERN', None, pattern)
matches = matcher(doc)

print_matches(matches)

Matches:  [beautiful design, smart search, automatic labels, optional voice responses]
